In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("../student_flow_coordinates.csv")

In [8]:
df = df.groupby(df.columns.tolist()).size().reset_index().rename(columns={0:'count'})

In [9]:
df = df.drop(columns=["Name of Organisation_send", "Name of Organisation_host"])

In [10]:
df = df.rename(columns={"coordinates_send": "send", "coordinates_host": "receive"})

In [11]:
# latitude longtitude

In [12]:
df["sendLat"] = df.send.apply(lambda x: x.split(",")[0].lstrip("(").strip())
df["sendLon"] = df.send.apply(lambda x: x.split(",")[1].rstrip(")").strip())

In [13]:
df["receiveLat"] = df.receive.apply(lambda x: x.split(",")[0].lstrip("(").strip())
df["receiveLon"] = df.receive.apply(lambda x: x.split(",")[1].rstrip(")").strip())

In [14]:
df = df.drop(columns=["send", "receive"])

In [23]:
ci = pd.read_csv("../geonames.org/countryInfo.txt", delimiter="\t")
ci = ci[["ISO", "ISO-Numeric"]].rename(columns={"Country": "name", "ISO-Numeric": "numeric"})

In [25]:
df = df.merge(ci, left_on="SendingCountry", right_on="ISO", how="left").drop(columns=["ISO"])\
        .rename(columns={"numeric": "sendingNumeric"})
df = df.merge(ci, left_on="ReceivingCountry", right_on="ISO", how="left").drop(columns=["ISO"])\
        .rename(columns={"numeric": "receivingNumeric"})

In [27]:
df = df.drop(columns=["SendingCountry", "ReceivingCountry"])

In [29]:
def padleft(x):
    while len(x) < 3:
        x = "0" + x
    return x

df.sendingNumeric = df.sendingNumeric.astype(str).apply(lambda x: padleft(x))
df.receivingNumeric = df.receivingNumeric.astype(str).apply(lambda x: padleft(x))

In [31]:
df.to_csv("../../src/data/arrowmap/coordinates.csv", index=False)